# Seinfeld Script Scraper 

### URL : http://www.seinology.com/scripts.shtml

In [1]:
from lxml import html
from json import dump,loads
from requests import get
import json
import re
from dateutil import parser as dateparser
from time import sleep
import pandas as pd
from itertools import chain
import numpy as np
from bs4 import BeautifulSoup
import urllib.request

In [2]:
def parse_sein_url(url):
    texts = ""
    response_ep = get(url)
    cleaned_response_ep = response_ep.text.replace('\x00', "")
    parser_ep = html.fromstring(cleaned_response_ep)
    return parser_ep.xpath(".//p//font//text()")

def clean_text(text):
    episode_info = {}
    series_text = pd.Series(text).str.replace("\xa0","")
    clean_text = series_text.str.replace("(\n|\t)","")
    episode_info["title"] = clean_text[0]
    episode_info["chronology"] = clean_text[1]
    episode_info["date"] = clean_text[2]
    episode_info["etc"] = clean_text[3]
    if episode_info["etc"] != '':
        if episode_info["title"] == 'Episode 45 - The Wallet (1)':
            episode_info["writer"] = clean_text[6]
            episode_info["director"] = clean_text[7]
        else:
            episode_info["writer"] = clean_text[5]
            episode_info["director"] = clean_text[6]
    else:
        episode_info["writer"] = clean_text[4]
        episode_info["director"] = clean_text[5]
    return episode_info

def extract_text(text):
    series_text = pd.Series(text)
    clean_text = series_text.str.replace("(\xa0|\n|\t)","")
    clean_text = clean_text.str.replace("\x91","'")
    clean_text = clean_text.str.replace("\x92","'")
    clean_text = clean_text.str.replace("\x93",'""')
    clean_text = clean_text.str.replace("\x94",'""')
    clean_text = clean_text.str.replace("\x96",'-')
    start_script = clean_text[clean_text.str.contains("====")].index[0] + 1
    lines = clean_text.iloc[start_script:]
    clean_lines = lines[lines != ""]
    return clean_lines.tolist()

def extract_cast_lines(a_list,cast_member):
    cast_member = cast_member.upper()
    if cast_member == "KRAMER":
        return [i for i in a_list if "{0}:".format(cast_member) in i or "KESSLER:" in i]
    elif cast_member == "ETC":
        return [i for i in a_list if ("GEORGE:" not in i) and ("ELAINE:") not in i and ("JERRY:" not in i) and ("KRAMER:" not in i) and ("KESSLER:" not in i) and ("NEWMAN:" not in i)]
    else:
        return [i for i in a_list if "{0}:".format(cast_member) in i]

In [3]:
response = get("http://www.seinology.com/scripts-english.shtml")
cleaned_response = response.text.replace('\x00', "")

parser = html.fromstring(cleaned_response)

episodes = {}
regexp = re.compile(r'\d.shtml')
for i in parser.xpath('.//a'):
    try:
        if regexp.search(i.attrib["href"]):
            if i.text == None:
                episodes[i.xpath(".//font")[0].text] = "http://www.seinology.com/{0}".format(i.attrib["href"])
            else:
                episodes[i.text] = "http://www.seinology.com/{0}".format(i.attrib["href"])
    except:
        x = 1
        
df = pd.DataFrame(list(episodes.items()),columns=["Episode","URL"])

In [4]:
df["text"] = df["URL"].apply(parse_sein_url)
text_info = pd.DataFrame(df["text"].apply(clean_text))["text"].apply(pd.Series)

dat = df.join(text_info)

In [5]:
dat["lines"] = dat["text"].apply(extract_text)

In [8]:
dat["jerry_lines"] = dat["lines"].apply(lambda x : extract_cast_lines(x,"JERRY"))
dat["elaine_lines"] = dat["lines"].apply(lambda x : extract_cast_lines(x,"ELAINE"))
dat["kramer_lines"] = dat["lines"].apply(lambda x : extract_cast_lines(x,"KRAMER"))
dat["george_lines"] = dat["lines"].apply(lambda x : extract_cast_lines(x,"GEORGE"))
dat["newman_lines"] = dat["lines"].apply(lambda x : extract_cast_lines(x,"NEWMAN"))
dat["etc_lines"] = dat["lines"].apply(lambda x : extract_cast_lines(x,"etc"))

In [9]:
dat

,Episode,URL,text,title,chronology,date,etc,writer,director,lines,jerry_lines,elaine_lines,kramer_lines,george_lines,newman_lines,etc_lines
0,"01-Good News, Bad News",http://www.seinology.com//scripts/script-01.shtml,"[Episode 1 - Good News, Bad News, \n\t\t\t\t\t...","Episode 1 - Good News, Bad News","pc: 101, season 1, episode 1 (Pilot)","Broadcast date: July 5, 1989","(The series is titled The Seinfeld Chronicles,...",Written By Larry David & Jerry Seinfeld,Directed By Art Wolff,"[INT. COMEDY CLUB - NIGHT, (Jerry is on stage,...",[JERRY: Do you know what this is all about? Do...,[],"[KESSLER: Are you up?, KESSLER: Boy, the Mets ...","[GEORGE: Are you through? , GEORGE: Yes, it wa...",[],"[INT. COMEDY CLUB - NIGHT, (Jerry is on stage,..."
1,02-The Stakeout,http://www.seinology.com//scripts/script-02.shtml,"[Episode 2 - The Stakeout, \n\t\t\t\t\t\t\t\n\...",Episode 2 - The Stakeout,"pc: 103, season 1, episode 1","Broadcast date: May 31, 1990",,Written By Larry David & Jerry Seinfeld,Directed By Tom Cherones,"[INT. COMEDY CLUB - NIGHT, (Jerry is on stage,...","[JERRY: So, I'm on line at the supermarket. Tw...","[ELAINE: """"Coccoon II: The Return"""". I guess t...","[KRAMER: Hi., KRAMER: Hey Morty! (to Jerry) Sa...","[GEORGE: What does she look like?, GEORGE: Wha...",[],"[INT. COMEDY CLUB - NIGHT, (Jerry is on stage,..."
2,03-The Robbery,http://www.seinology.com//scripts/script-03.shtml,"[Episode 3 - The Robbery, \n\t\t\t\t\t\t\t\n\t...",Episode 3 - The Robbery,"pc: 104, season 1, episode 2","Broadcast date: June 7, 1990",,Written By Matt Goldman,Directed By Tom Cherones,"[INT. COMEDY CLUB, (Jerry is on stage, perform...","[JERRY: So I move into the centre lane, now I ...",[ELAINE: (claps her hands) That is the single ...,"[KRAMER: Uh, Jer, well you know, I was cookin'...","[GEORGE: (from the intercom) Yeah., GEORGE: (t...",[],"[INT. COMEDY CLUB, (Jerry is on stage, perform..."
3,04-Male Unbonding,http://www.seinology.com//scripts/script-04.shtml,"[Episode 4 - Male Unbonding, \n\t\t\t\t\t\t\t\...",Episode 4 - Male Unbonding,"pc: 102, season 1, episode 3","Broadcast date: June 14, 1990",,Written By Larry David & Jerry Seinfeld,Directed By Tom Cherones,"[INT. COMEDY CLUB - NIGHT, (Jerry is on stage,...",[JERRY: Most men like working on things. Tools...,"[ELAINE: Come on, let's go do something. I don...","[KRAMER: (to the phone) Oh, they've got a cure...",[GEORGE: I had to say something. (chuckles) I ...,[],"[INT. COMEDY CLUB - NIGHT, (Jerry is on stage,..."
4,05-The Stock Tip,http://www.seinology.com//scripts/script-05.shtml,"[Episode 5 - The Stock Tip, \n\t\t\t\t\t\t\t\n...",Episode 5 - The Stock Tip,"pc: 105, season 1, episode 4","Broadcast date: June 21, 1990",,Written By Larry David & Jerry Seinfeld,Directed By Tom Cherones,"[INT. COMEDY CLUB, (Jerry is on stage, perform...",[JERRY: Went out to dinner the other night. Ch...,"[ELAINE: I know, I know. I'm sorry I'm late., ...","[KRAMER: Bad news, my friend., KRAMER: Two and...",[GEORGE: I never heard him say anything really...,[],"[INT. COMEDY CLUB, (Jerry is on stage, perform..."
5,06-The Ex-Girlfriend,http://www.seinology.com//scripts/script-06.shtml,"[Episode 6 - The Ex-Girlfriend, \n\t\t\t\t\t\t...",Episode 6 - The Ex-Girlfriend,"pc: 201, season 2, episode 1","Broadcast date: January 16, 1991",,Written By Larry David & Jerry Seinfeld,Directed By Tom Cherones,"[INT. COMEDY CLUB - NIGHT, (Jerry is on stage,...",[JERRY: I'm always in traffic with the lane ex...,"[ELAINE: Hi., ELAINE: Hey, what are you doing?...","[KRAMER: (to the intercom) Come on up., KRAMER...","[GEORGE: She can't kill me right?, GEORGE: Peo...",[],"[INT. COMEDY CLUB - NIGHT, (Jerry is on stage,..."
6,07-The Pony Remark,http://www.seinology.com//scripts/script-07.shtml,"[Episode 7 - The Pony Remark, \n\t\t\t\t\t\t\t...",Episode 7 - The Pony Remark,"pc: 202, season 2, episode 2","Broadcast date: January 30, 1991",,Written By Larry David & Jerry Seinfeld,Directed By Tom Cherones,"[INT. COMEDY CLUB - NIGHT, (J